# Busca em Largura


## 📘 1. Introdução

A Busca em Largura é uma estratégia de busca sem informação utilizada para explorar todos os nós de um grafo de forma sistemática, nível por nível. Ela utiliza uma fila (FIFO) como estrutura de dados principal e garante a descoberta do caminho mais curto (em número de passos) em grafos não ponderados.

### Características:
- Estratégia: Primeiro a Entrar, Primeiro a Sair (FIFO)
- Completa: Sim
- Ótima: Sim, para grafos com custo uniforme
- Complexidade de tempo: $O(b^d)$, onde $b$ é o fator de ramificação e $d$ a profundidade da solução
- Complexidade de espaço: $O(b^d)$

## 📘 2. Estrutura Básica do Problema

### Representação dos nós (estados)

In [1]:
class Estado:
    """
    Representa um estado no espaço de busca de um problema.

    A estrutura é genérica, podendo ser utilizada em problemas como o quebra-cabeça de 8 peças,
    labirintos, jogos e outros domínios que envolvem transições de estado.

    A representação do estado é feita por meio de um vetor unidimensional.
    """

    def __init__(self, vetor=None, custo=0, acao=None, pai=None):
        """
        Inicializa um novo estado.

        Args:
            vetor (list[int], opcional): representação linear do estado (ex: 9 elementos do 8-puzzle).
            custo (int): custo acumulado para atingir este estado a partir do estado inicial.
            acao (str): ação que resultou neste estado a partir do estado pai (ex: 'cima', 'direita').
            pai (Estado): referência ao estado anterior (para reconstrução do caminho da solução).
        """
        if vetor is None:
            vetor = []
        self.vetor = vetor
        self.custo = custo
        self.acao = acao
        self.pai = pai

    def __getitem__(self, i):
        """
        Permite o acesso direto aos elementos do vetor via indexação.

        Args:
            i (int): índice do elemento a ser acessado.

        Returns:
            int: valor na posição i do vetor.
        """
        return self.vetor[i]

    def __eq__(self, outro):
        """
        Define igualdade entre dois estados com base em seus vetores.

        Args:
            outro (Estado): estado a ser comparado.

        Returns:
            bool: True se os vetores forem iguais, False caso contrário.
        """
        if not isinstance(outro, Estado):
            return False
        return self.vetor == outro.vetor

    def __hash__(self):
        """
        Permite o uso de instâncias de Estado como chaves de dicionário ou em conjuntos.

        Returns:
            int: valor hash derivado do vetor (convertido em tupla).
        """
        return hash(tuple(self.vetor))

    def __repr__(self):
        """
        Retorna uma representação legível do vetor do estado, útil para debug.

        Returns:
            str: representação textual do vetor do estado.
        """
        return str(self.vetor)


### Representação do Problema

In [2]:
class Problema:
    """
    Classe base para a modelagem de problemas de busca no espaço de estados.

    Esta classe define a interface e estrutura mínima que deve ser seguida por qualquer
    problema a ser resolvido com algoritmos de busca, como Busca em Largura, A*, entre outros.
    """

    def __init__(self):
        """
        Inicializa o problema com estado inicial nulo.
        Subclasses devem sobrescrever essa inicialização.
        """
        self._estado_inicial = None

    @property
    def estado_inicial(self):
        """
        Retorna o estado inicial do problema.

        Returns:
            Estado: o estado de partida.
        """
        if self._estado_inicial is None:
            raise NotImplementedError("O estado inicial não foi definido.")
        return self._estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado fornecido atende à condição de objetivo.

        Args:
            estado (Estado): estado a ser testado.

        Returns:
            bool: True se é um estado objetivo, False caso contrário.
        """
        raise NotImplementedError("Método 'estado_objetivo' deve ser implementado na subclasse.")

    def solucao(self, estado):
        """
        Reconstrói o caminho do estado inicial até o estado objetivo, percorrendo os pais.

        Args:
            estado (Estado): estado objetivo atingido.

        Returns:
            list[Estado]: sequência de estados do caminho da solução.
        """
        resultado = []
        ptr = estado
        while ptr:
            resultado.append(ptr)
            ptr = ptr.pai
        return list(reversed(resultado))

    def funcao_sucessora(self, estado):
        """
        Retorna a lista de estados sucessores (vizinhos) do estado atual,
        aplicando todas as ações válidas possíveis.

        Args:
            estado (Estado): estado atual.

        Returns:
            list[Estado]: estados sucessores gerados.
        """
        raise NotImplementedError("Método 'funcao_sucessora' deve ser implementado na subclasse.")


## 📘 3. Implementação da Busca em Largura

In [3]:
def busca_largura(problema: Problema):
    """
    Executa o algoritmo de Busca em Largura (Breadth-First Search - BFS)
    para resolver um problema de busca no espaço de estados.

    Args:
        problema (Problema): Instância que define o problema com os métodos
                             estado_inicial, estado_objetivo e funcao_sucessora.

    Returns:
        list[Estado]: Caminho da solução, do estado inicial ao objetivo.

    Raises:
        RuntimeError: Se a borda for esvaziada sem encontrar uma solução.
    """

    # 1. Inicializa a borda com o estado inicial
    borda = [problema.estado_inicial]  # fila FIFO

    # 2. Inicializa a memória com o estado inicial visitado
    memoria = [problema.estado_inicial]

    # 3. Loop principal da busca
    while borda:

        # 4. Remove o primeiro estado da borda
        estado = borda.pop(0)

        # 5. Verifica se é o estado objetivo
        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        # 6. Gera sucessores do estado atual
        vizinhos = problema.funcao_sucessora(estado)

        # 7. Adiciona novos estados à borda e à memória
        for vizinho in vizinhos:
            if vizinho not in memoria:
                borda.append(vizinho)
                memoria.append(vizinho)

    # 8. Se a borda esvaziar, a solução não foi encontrada
    raise RuntimeError("Falha ao encontrar solução.")


## 📘 4. Exemplo - Quebra cabeça de 8

In [4]:
from copy import deepcopy

class ProblemaQuebraCabeca8(Problema):
    """
    Implementação do problema do Quebra-Cabeça de 8 (8-puzzle) como subclasse de Problema.

    O estado é representado como um vetor linear de 9 posições, onde:
        - Cada posição representa uma célula da matriz 3x3.
        - O valor 0 representa o espaço vazio.

    O objetivo é alcançar a configuração:
        [1, 2, 3,
         4, 5, 6,
         7, 8, 0]
    """

    def __init__(self, estado_inicial=None):
        """
        Inicializa o problema com um estado inicial opcional.
        Se não for fornecido, usa um estado padrão.

        Args:
            estado_inicial (Estado): estado inicial do problema (opcional).
        """
        if estado_inicial is None:
            vetor_inicial = [1, 2, 3,
                             4, 0, 5,
                             6, 7, 8]
            estado_inicial = Estado(vetor=vetor_inicial)
        self._estado_inicial = estado_inicial

    def estado_objetivo(self, estado):
        """
        Verifica se o estado atual corresponde ao objetivo.

        Args:
            estado (Estado): estado atual.

        Returns:
            bool: True se o estado é o objetivo.
        """
        return estado.vetor == [1, 2, 3, 4, 5, 6, 7, 8, 0]

    def funcao_sucessora(self, estado):
        """
        Gera os estados vizinhos a partir do estado atual,
        movendo o espaço vazio (0) em até 4 direções possíveis.

        Args:
            estado (Estado): estado atual.

        Returns:
            list[Estado]: lista de estados sucessores.
        """
        sucessores = []
        idx = estado.vetor.index(0)  # posição do espaço vazio
        linha, coluna = divmod(idx, 3)

        # Direções possíveis com seus deslocamentos
        movimentos = {
            'cima':    (-1,  0),
            'baixo':   (+1,  0),
            'esquerda':( 0, -1),
            'direita': ( 0, +1)
        }

        for acao, (dx, dy) in movimentos.items():
            nova_linha = linha + dx
            nova_coluna = coluna + dy

            if 0 <= nova_linha < 3 and 0 <= nova_coluna < 3:
                novo_idx = nova_linha * 3 + nova_coluna
                novo_vetor = deepcopy(estado.vetor)
                # Troca o 0 com o valor adjacente
                novo_vetor[idx], novo_vetor[novo_idx] = novo_vetor[novo_idx], novo_vetor[idx]
                novo_estado = Estado(vetor=novo_vetor,
                                     custo=estado.custo + 1,
                                     acao=acao,
                                     pai=estado)
                sucessores.append(novo_estado)

        return sucessores

In [5]:
# Execução do problema
problema = ProblemaQuebraCabeca8()
caminho = busca_largura(problema)

# Impressão simples dos estados no caminho
for passo, estado in enumerate(caminho):
    print(f"Passo {passo} - Ação: {estado.acao}")
    for i in range(0, 9, 3):
        print(estado.vetor[i:i+3])
    print()

Passo 0 - Ação: None
[1, 2, 3]
[4, 0, 5]
[6, 7, 8]

Passo 1 - Ação: direita
[1, 2, 3]
[4, 5, 0]
[6, 7, 8]

Passo 2 - Ação: baixo
[1, 2, 3]
[4, 5, 8]
[6, 7, 0]

Passo 3 - Ação: esquerda
[1, 2, 3]
[4, 5, 8]
[6, 0, 7]

Passo 4 - Ação: esquerda
[1, 2, 3]
[4, 5, 8]
[0, 6, 7]

Passo 5 - Ação: cima
[1, 2, 3]
[0, 5, 8]
[4, 6, 7]

Passo 6 - Ação: direita
[1, 2, 3]
[5, 0, 8]
[4, 6, 7]

Passo 7 - Ação: baixo
[1, 2, 3]
[5, 6, 8]
[4, 0, 7]

Passo 8 - Ação: direita
[1, 2, 3]
[5, 6, 8]
[4, 7, 0]

Passo 9 - Ação: cima
[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

Passo 10 - Ação: esquerda
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

Passo 11 - Ação: esquerda
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

Passo 12 - Ação: baixo
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

Passo 13 - Ação: direita
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Passo 14 - Ação: direita
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



## 📘 5. Exemplo Visual

In [6]:
!pip install graphviz

In [7]:
from graphviz import Digraph
from IPython.display import display
import time

def busca_largura_visual(problema: Problema, atraso=1.0):
    """
    Executa busca em largura com visualização gráfica da árvore.
    """
    borda = [problema.estado_inicial]
    memoria = [problema.estado_inicial]

    grafo = Digraph(format='png')
    grafo.attr('graph', rankdir='TB')  # top-to-bottom
    grafo.attr('node',
               shape='box',
               style='filled,rounded',
               fontname='monospace',
               fontsize='10',
               width='0.4',
               height='0.4',
               margin='0.05')

    grafo.attr('edge', fontname='monospace', fontsize='10')

    id_nos = {}
    contador_id = 0

    def estado_id(estado):
        nonlocal contador_id
        if estado not in id_nos:
            id_nos[estado] = f"n{contador_id}"
            contador_id += 1
        return id_nos[estado]

    def formatar_matriz(vetor):
        """Formata vetor como matriz 3x3 para exibição no nó."""
        return '\n'.join([
            ' '.join(f"{x}" if x != 0 else " " for x in vetor[i:i+3])
            for i in range(0, 9, 3)
        ])

    while borda:
        estado = borda.pop(0)
        id_estado = estado_id(estado)

        # Destaca o estado atual com amarelo
        grafo.node(id_estado, formatar_matriz(estado.vetor), fillcolor='gold')

        # Exibe árvore parcial
        display(grafo)
        time.sleep(atraso)

        if problema.estado_objetivo(estado):
            return problema.solucao(estado)

        vizinhos = problema.funcao_sucessora(estado)

        for vizinho in vizinhos:
            id_pai = estado_id(estado)
            id_filho = estado_id(vizinho)

            if vizinho not in memoria:
                memoria.append(vizinho)
                borda.append(vizinho)

                # Nó filho em verde
                grafo.node(id_filho, formatar_matriz(vizinho.vetor), fillcolor='palegreen')
                grafo.edge(id_pai, id_filho, label=vizinho.acao)

        # Estado já visitado fica em cinza escuro
        grafo.node(id_estado, formatar_matriz(estado.vetor), fillcolor='dimgray')

    raise RuntimeError("Falha ao encontrar solução.")


In [8]:
# Descomente o código abaixo para visualizar a execução do algoritmo

#problema = ProblemaQuebraCabeca8()
#caminho = busca_largura_visual(problema, atraso=0.8)